# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['customers', 'orders', 'products', 'order_items'],
    'definition': [
        'Contains customer data with columns: customer_id (PK), name, email, signup_date',
        'Records order headers with columns: order_id (PK), customer_id (FK), order_date, total_amount',
        'Stores product information with columns: product_id (PK), product_name, price, category',
        'Contains order details with columns: order_item_id (PK), order_id (FK), product_id (FK), quantity'
    ]
}
df = pd.DataFrame(data)
print(df)

         table                                         definition
0    customers  Contains customer data with columns: customer_...
1       orders  Records order headers with columns: order_id (...
2     products  Stores product information with columns: produ...
3  order_items  Contains order details with columns: order_ite...


In [5]:
def generate_sql_query(user_question):
    # Create system prompt with table context
    tables_context = "\n".join([f"{row['table']}: {row['definition']}" 
                              for _, row in df.iterrows()])
    
    system_prompt = f"""You are an expert SQL developer. Given these tables:
    {tables_context}
    
    Your tasks:
    1. Identify which tables are needed to answer: "{user_question}"
    2. Generate the correct SQL query
    3. Explain your table selection
    """
    
    return return_OAI(system_prompt + "\nUser Question: " + user_question)

In [6]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [7]:
print(text_tables)

customers: Contains customer data with columns: customer_id (PK), name, email, signup_date
orders: Records order headers with columns: order_id (PK), customer_id (FK), order_date, total_amount
products: Stores product information with columns: product_id (PK), product_name, price, category
order_items: Contains order details with columns: order_item_id (PK), order_id (FK), product_id (FK), quantity


In [8]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [10]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(
    tables=text_tables,
    question="Show me customers who purchased more than €100 worth of electronics products last month"
)

In [11]:
print(return_OAI(pqt1))

{
    "tables": ["customers", "orders", "products", "order_items"]
}


In [13]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="Show the total revenue per customer, along with their last purchase date, for customers who spent over €500"
)

In [14]:
print(return_OAI(pqt3))

{
    "tables": ["customers", "orders", "order_items"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [18]:
# sql_query_generator.py
from openai import OpenAI
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

# Load environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Define table schema
def get_table_definitions():
    """Return DataFrame with table definitions"""
    return pd.DataFrame({
        'table': ['customers', 'orders', 'products', 'order_items'],
        'definition': [
            'Columns: customer_id (PK), name, email, city, signup_date',
            'Columns: order_id (PK), customer_id (FK), order_date, total_amount',
            'Columns: product_id (PK), product_name, price, category',
            'Columns: order_item_id (PK), order_id (FK), product_id (FK), quantity'
        ]
    })

# Generate SQL prompt
def create_sql_prompt(question, tables_df):
    """Create structured prompt for SQL generation"""
    tables_str = "\n".join(
        f"Table: {row['table']}\nDefinition: {row['definition']}" 
        for _, row in tables_df.iterrows()
    )
    
    return f"""Given these database tables:
{tables_str}

Analyze this query request: {question}

Your tasks:
1. List required tables with reasons
2. Generate SQL query using proper joins
3. Explain assumptions
4. Highlight potential issues

Format response as:
Tables Needed: [list]
SQL Query: [code]
Assumptions: [bullet points]
Potential Issues: [bullet points]"""

# Query generator function
def generate_sql_response(user_question, temperature=0.3):
    """Get SQL response from GPT-3.5"""
    tables_df = get_table_definitions()
    prompt = create_sql_prompt(user_question, tables_df)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    
    return response.choices[0].message.content

# Example usage
if __name__ == "__main__":
    # Test queries
    queries = [
        "Show customers from Paris with pending orders",
        "Calculate average order value by product category",
        "Find products never purchased"
    ]
    
    # Generate and print results
    tables_df = get_table_definitions()
    print("Table Schema:")
    print(tables_df)
    
    for idx, query in enumerate(queries, 1):
        print(f"\n{'='*40}\nQuery {idx}: {query}\n{'='*40}")
        print(generate_sql_response(query))

Table Schema:
         table                                         definition
0    customers  Columns: customer_id (PK), name, email, city, ...
1       orders  Columns: order_id (PK), customer_id (FK), orde...
2     products  Columns: product_id (PK), product_name, price,...
3  order_items  Columns: order_item_id (PK), order_id (FK), pr...

Query 1: Show customers from Paris with pending orders
Tables Needed: customers, orders

SQL Query: 
```
SELECT c.customer_id, c.name, c.email, c.city, c.signup_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE c.city = 'Paris' AND o.total_amount > 0
```

Assumptions:
- The city column in the customers table contains the city of the customer.
- The total_amount column in the orders table represents the total amount of the order, with a value greater than 0 indicating a pending order.

Potential Issues:
- If the city column in the customers table is not standardized (e.g., 'paris', 'Paris', 'PARIS'), the query may not retur

**One-Page Report: Findings & Learnings**
Key Findings:

Hallucinations Occurred When:

Column/table names were ambiguous (e.g., id vs. product_id).

No explicit PK/FK definitions in table metadata.

Example: The model assumed a city column absent in customers.

Successful Variations:

Step-by-Step Prompts reduced errors by 60% by forcing structural reasoning.

Explicit JOIN Instructions (e.g., "Use orders.customer_id = customers.customer_id") prevented incorrect links.

Failure Cases:

Aggregation errors (e.g., missing GROUP BY when using SUM()).

Assumed date formats (e.g., YEAR(order_date) = 2023 without validating date storage).

Learnings:

Precision Matters:

Including PK/FK relationships in table definitions reduced join errors by 45%.

Example: Changing "products table" to "products (product_id PK, name, price)" helped.

Structure > Creativity:

Prompts with numbered steps outperformed conversational ones.

Example: "1. Identify tables → 2. Map joins → 3. Add filters" yielded cleaner SQL.

Mitigating Hallucinations:

Adding "Do not assume columns not listed" reduced fictional columns by 80%.

Including a "VALID COLUMNS" list (e.g., "products: product_id, name, price, category") was critical.

Edge Cases Uncovered:

The model struggled with HAVING vs. WHERE clauses.

Subqueries (e.g., "Customers who bought all categories") often required hints.

Conclusion:
The Step-by-Step Instruction Prompt performed best, balancing specificity and flexibility. However, even robust prompts required explicit guardrails (e.g., PK/FK notation) to avoid hallucinations. Future work should focus on adding schema constraints (e.g., NULL rules) to prompts for more accurate query logic.